In [54]:
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.metrics import confusion_matrix, accuracy_score
from scipy.stats import mode

import pandas as pd
import numpy as np

In [55]:
iris = load_iris()
df = pd.DataFrame(iris.data, columns=iris.feature_names)
df['class'] = iris.target 
print(df['class'].unique()) #CLass are [0,1,2] 3 classes 0 : Versicolor, 1 : Setosa, 2 : Virginica 
print("\n",df.info()) #sample is 150 entries, feature 4 data, and target 1 data.
print("\n",df.tail())

[0 1 2]
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   sepal length (cm)  150 non-null    float64
 1   sepal width (cm)   150 non-null    float64
 2   petal length (cm)  150 non-null    float64
 3   petal width (cm)   150 non-null    float64
 4   class              150 non-null    int32  
dtypes: float64(4), int32(1)
memory usage: 5.4 KB

 None

      sepal length (cm)  sepal width (cm)  petal length (cm)  petal width (cm)  \
145                6.7               3.0                5.2               2.3   
146                6.3               2.5                5.0               1.9   
147                6.5               3.0                5.2               2.0   
148                6.2               3.4                5.4               2.3   
149                5.9               3.0                5.1               1.8   

     c

In [56]:
#Allow duplication of bagging 10 times to pull out data and make 20 samples. 
#A total of 200 data are created and data that have never been pulled out are stored in Out of data
def Bagging(dataset):
    bagging_df = pd.DataFrame(columns=dataset.columns)
    test_df = pd.DataFrame(columns=dataset.columns)
    Out_of_Data = set(range(dataset.shape[0])) #Initial 200 sample(0~199), not duplication 

    training_index = pd.DataFrame() #save bootstraped index.

    #generate 10 bagging dataset and each with 20 records.
    for i in range(10):
        data_index = list(range(150)) 
        random_data_index = np.random.choice(data_index, 20, replace=True) #restoration extraction
        training_index[f'{i}th'] = random_data_index
        bagging_sample = dataset.iloc[random_data_index, :]
        bagging_df = pd.concat([bagging_df, bagging_sample], axis=0) #Stored in bagging_df via pulled data indices
        Out_of_Data -= set(random_data_index) #remove the sampled data index
    
    Out_of_Data = list(Out_of_Data)
    test_df = dataset.iloc[Out_of_Data, :]
    print("\n Not sampled data index : ",Out_of_Data, "\n Not sampled data length is : ",len(Out_of_Data))
    print("\nThe bootstrap training dataset index\n", training_index)
    return bagging_df, test_df

In [57]:

#How to create #testdataset: Find the index as an intersection that has never been selected in the process of creating 10 bootstraps.
bagging_df, test_df = Bagging(df)

X_train = bagging_df.iloc[:,0:4].values
y_train = bagging_df.iloc[:,-1].values

X_test = test_df.iloc[:,0:4].values
y_test = test_df.iloc[:,-1].values

y_train = y_train.astype(int)



 Not sampled data index :  [3, 4, 12, 16, 17, 18, 19, 21, 23, 24, 28, 45, 48, 55, 57, 70, 79, 81, 84, 86, 90, 92, 95, 103, 109, 113, 116, 117, 118, 122, 126, 131, 133, 137, 140, 141, 149] 
 Not sampled data length is :  37

The bootstrap training dataset index
     0th  1th  2th  3th  4th  5th  6th  7th  8th  9th
0    31  132  111  111  102   49   51    0   74   94
1   146   96  124   62   74   47   76   67   40   53
2   111   74   36   67   52  132   80   20   87  123
3     0   75   68   30  130   98   72  129   78  110
4    15    8  136   22   91    5  144   54  102   67
5   135   10   42   82  135  145  128  119   63  124
6    13  121   46  128   80   69   82   26  142   15
7    65  123   85   88   85  104  145    1  125   44
8     0  112  134   49  114  136  132   78   97   31
9    43   93   98  139   35   13   35  120  106  112
10   47    6   32   40    7    9   14   27  145   75
11   68   59  145   73   67   50   53   41   49  105
12   63   99   39   25   93   69   71   96   34 

C:\Users\ksmin\AppData\Local\Temp\ipykernel_21268\2636670353.py:16: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  bagging_df = pd.concat([bagging_df, bagging_sample], axis=0) #Stored in bagging_df via pulled data indices


In [58]:
print(X_train)

[[5.4 3.4 1.5 0.4]
 [6.3 2.5 5.  1.9]
 [6.4 2.7 5.3 1.9]
 [5.1 3.5 1.4 0.2]
 [5.7 4.4 1.5 0.4]
 [7.7 3.  6.1 2.3]
 [4.3 3.  1.1 0.1]
 [6.7 3.1 4.4 1.4]
 [5.1 3.5 1.4 0.2]
 [5.  3.5 1.6 0.6]
 [4.6 3.2 1.4 0.2]
 [6.2 2.2 4.5 1.5]
 [6.1 2.9 4.7 1.4]
 [6.  3.  4.8 1.8]
 [6.7 3.1 4.4 1.4]
 [4.7 3.2 1.3 0.2]
 [6.3 2.3 4.4 1.3]
 [5.5 2.5 4.  1.3]
 [6.5 3.  5.8 2.2]
 [5.1 3.8 1.6 0.2]
 [6.4 2.8 5.6 2.2]
 [5.7 2.9 4.2 1.3]
 [6.4 2.9 4.3 1.3]
 [6.6 3.  4.4 1.4]
 [4.4 2.9 1.4 0.2]
 [5.4 3.7 1.5 0.2]
 [5.6 2.8 4.9 2. ]
 [6.3 2.7 4.9 1.8]
 [6.8 3.  5.5 2.1]
 [5.  2.3 3.3 1. ]
 [4.6 3.4 1.4 0.3]
 [5.2 2.7 3.9 1.4]
 [5.7 2.8 4.1 1.3]
 [5.7 2.8 4.1 1.3]
 [5.2 2.7 3.9 1.4]
 [5.7 2.9 4.2 1.3]
 [6.3 3.4 5.6 2.4]
 [6.7 3.1 4.4 1.4]
 [5.  2.3 3.3 1. ]
 [5.  2.3 3.3 1. ]
 [6.4 2.7 5.3 1.9]
 [6.7 3.3 5.7 2.1]
 [5.5 3.5 1.3 0.2]
 [6.2 2.2 4.5 1.5]
 [6.3 3.4 5.6 2.4]
 [4.4 3.2 1.3 0.2]
 [5.1 3.8 1.6 0.2]
 [6.  3.4 4.5 1.6]
 [6.1 2.6 5.6 1.4]
 [5.1 2.5 3.  1.1]
 [5.2 4.1 1.5 0.1]
 [6.7 3.  5.2 2.3]
 [5.1 3.4 1.

In [59]:
print(y_train, "\n y_train length is : ", len(y_train))
print("\n", y_test, "\n y_test length is : ", len(y_test))

[0 2 2 0 0 2 0 1 0 0 0 1 1 2 1 0 1 1 2 0 2 1 1 1 0 0 2 2 2 1 0 1 1 1 1 1 2
 1 1 1 2 2 0 1 2 0 0 1 2 1 0 2 0 1 0 0 0 1 2 2 2 1 1 0 0 1 2 1 0 2 0 1 0 0
 1 1 1 0 2 2 2 1 1 2 1 2 1 1 2 0 0 1 1 1 1 1 1 0 1 1 0 0 2 1 0 2 1 2 2 0 0
 1 1 0 2 0 1 1 0 2 1 1 1 1 2 2 1 2 2 0 0 1 1 1 2 0 1 2 1 0 0 1 0 2 1 2 0 0
 1 2 0 0 1 0 1 2 0 2 1 2 1 0 1 1 2 1 2 2 1 2 2 0 0 0 1 0 0 2 0 0 1 1 2 2 1
 2 0 0 0 2 1 2 0 1 0 0 2 0 0 2] 
 y_train length is :  200

 [0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2] 
 y_test length is :  37


In [68]:
def bagging_CF(X_train,X_test,y_train,y_test):
    #initialize 10 base learner model. and each base learners prediction about X_test
    bagging_models = []
    base_learner_predictions = np.zeros((X_test.shape[0],10))

    #Run 10 (training) bagging rounds with BaggingClassifier function.
    for i in range(10):
        X = []
        y= []
        bagging_clf = BaggingClassifier(DecisionTreeClassifier(), n_estimators=1, bootstrap=False)
        X = X_train[(20*i):(20*i+20)]
        y = y_train[(20*i):(20*i+20)]

        bagging_clf.fit(X, y)
        bagging_models.append(bagging_clf)

    #each 10 base learners predict X_test(not sampled)
    for i, model in enumerate(bagging_models):
        base_learner_predictions[:,i] = model.estimators_[0].predict(X_test)
    
    # Perform majority voting (mode[0] : Most frequently repeated values, mode[1] : Repeated frequency)
    majority_vote_predictions, _ = mode(base_learner_predictions, axis=1)
    majority_vote_predictions = majority_vote_predictions.flatten()

    y_pred = majority_vote_predictions

    #accuracy score between real data and majority voted data = ensembled data
    print(accuracy_score(y_test, y_pred))

    #Show a table (dataframe) with predictions from the 10 base learners and the final ensemble learner
    predictions_df = pd.DataFrame(base_learner_predictions, columns=[f'{i+1}' for i in range(10)])

    predictions_df['Ensemble'] = majority_vote_predictions
    predictions_df["Reality"] = y_test

    confusion_matrix = Confuse_Matrix(predictions_df['Reality'],predictions_df['Ensemble'])
    return predictions_df,confusion_matrix

#Show the confusion matrix
def Confuse_Matrix(reality, ensemble):
    confusion_matrix = pd.crosstab(reality,ensemble,rownames=['Actual'], colnames=['Predicted'])

    return confusion_matrix

In [69]:
predictions_df,confusion_matrix = bagging_CF(X_train,X_test,y_train,y_test)

print(predictions_df)

0.972972972972973
      1    2    3    4    5    6    7    8    9   10  Ensemble  Reality
0   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0       0.0        0
1   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0       0.0        0
2   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0       0.0        0
3   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0       0.0        0
4   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0       0.0        0
5   0.0  0.0  0.0  1.0  0.0  0.0  0.0  1.0  0.0  0.0       0.0        0
6   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0       0.0        0
7   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0       0.0        0
8   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0       0.0        0
9   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0       0.0        0
10  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0       0.0        0
11  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0       0.0        0
12  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.

In [70]:
print(confusion_matrix)

Predicted  0.0  1.0  2.0
Actual                  
0           13    0    0
1            0    9    1
2            0    0   14
